In [1]:
#Import Libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import os
from torch.utils.data import TensorDataset, DataLoader

# Load dataset

In [2]:
# Load preprocessed data
train_data = pd.read_csv('data/electricity_consump/electricity_train.csv')
test_data = pd.read_csv('data/electricity_consump/electricity_test.csv')

# View the first 5 rows of training set
train_data.head()

,timestamp,consumption
0,2011-01-01 00:15:00,-0.704319
1,2011-01-01 00:30:00,-0.704319
2,2011-01-01 00:45:00,-0.678983
3,2011-01-01 01:00:00,-0.653647
4,2011-01-01 01:15:00,-0.704319


In [3]:
print(train_data.shape)
print(test_data.shape)

(105215, 2)
(35040, 2)


In [4]:
def create_sequences(df, seq_length):
    xs, ys = [], []
    # Iterate over data indices
    for i in range(len(df)- seq_length):
      	# Define inputs
        x = df.iloc[i:(i+seq_length), 1]
        # Define target
        y = df.iloc[i+seq_length, 1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

In [36]:
# Use create_sequences to create inputs and targets
seq_length=96
X_train, y_train = create_sequences(train_data, seq_length)
print(X_train.shape, y_train.shape)

print(X_train[0], y_train[0])

(105119, 96) (105119,)
[-0.70431852 -0.70431852 -0.67898263 -0.65364675 -0.70431852 -0.70431852
 -0.72990776 -0.70431852 -0.70431852 -0.67898263 -0.67898263 -0.62831087
 -0.67898263 -0.72990776 -0.72990776 -0.72990776 -0.70431852 -0.67898263
 -0.70431852 -0.75524365 -0.67898263 -0.67898263 -0.67898263 -0.65364675
 -0.72990776 -0.70431852 -0.67898263 -0.67898263 -0.70431852 -0.72990776
 -0.70431852 -0.72990776 -0.67898263 -0.70431852 -0.75524365 -0.90725895
 -1.00885585 -0.95793072 -0.93259484 -0.98326661 -0.98326661 -1.00885585
 -0.98326661 -1.00885585 -0.98326661 -0.95793072 -0.95793072 -0.95793072
 -0.95793072 -1.00885585 -0.98326661 -0.95793072 -0.98326661 -0.98326661
 -1.00885585 -0.95793072 -0.93259484 -1.00885585 -0.98326661 -0.93259484
 -0.98326661 -1.03419173 -0.98326661 -0.98326661 -0.85658718 -0.67898263
 -0.72990776 -0.72990776 -0.70431852 -0.70431852 -0.72990776 -0.70431852
 -0.70431852 -0.67898263 -0.67898263 -0.65364675 -0.67898263 -0.65364675
 -0.67898263 -0.70431852 -0.

In [43]:
#convert X_train and y_train to tensors
X_train = torch.tensor(X_train).float()
print(X_train.shape)

torch.Size([105119, 96])


In [44]:
y_train = torch.tensor(y_train).float()
print(y_train.shape)

torch.Size([105119])


In [42]:
# Use create_sequences to create inputs and targets
seq_length=96
X_train, y_train = create_sequences(train_data, seq_length)
print(X_train.shape, y_train.shape)

# Create TensorDataset
dataset_train = TensorDataset(
    torch.from_numpy(X_train).float(),
    torch.from_numpy(y_train).float(),
)
print(len(dataset_train))

(105119, 96) (105119,)
105119


In [17]:
#Create dataloader with training data
train_loader = DataLoader(dataset_train, batch_size=32, shuffle=False)
for X_batch, y_batch in train_loader:
    print(X_batch.shape, y_batch.shape)
    X_batch_reshape = X_batch.view(32,96,1)
    print(X_batch_reshape.shape)
    break

torch.Size([32, 96]) torch.Size([32])
torch.Size([32, 96, 1])


In [24]:
#Building a forecasting RNN model
class RNN(nn.Module):
    def __init__(self):
        super().__init__()
        # Define RNN layer
        self.rnn = nn.RNN(
            input_size=1,
            hidden_size=32,
            num_layers=2,
            batch_first=True,
        )
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        # Initialize first hidden state with zeros
        h0 = torch.zeros(2, x.size(0), 32)
        # Pass x and h0 through recurrent layer
        out, _ = self.rnn(x, h0)  
        # Pass recurrent layer's last output through linear layer
        out = self.fc(out[:, -1, :])
        return out

In [25]:
print(RNN())

RNN(
  (rnn): RNN(1, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)


In [9]:
#Building LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        # Define lstm layer
        self.lstm = nn.LSTM(
            input_size=1,
            hidden_size=32,
            num_layers=2,
            batch_first=True,
        )
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 32)
        # Initialize long-term memory
        c0 = torch.zeros(2, x.size(0), 32)
        # Pass all inputs to lstm layer
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [11]:
print(LSTM(input_size = 96))

LSTM(
  (lstm): LSTM(1, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)


In [12]:
#Building GRU model
class GRU(nn.Module):
    def __init__(self):
        super().__init__()
        # Define RNN layer
        self.gru = nn.GRU(
            input_size=1,
            hidden_size=32,
            num_layers=2,
            batch_first=True,
        )
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 32)
        out, _ = self.gru(x, h0)  
        out = self.fc(out[:, -1, :])
        return out

In [13]:
print(GRU())

GRU(
  (gru): GRU(1, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)


In [31]:
#Test model
test_model = RNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(test_model.parameters(), lr=0.001)

#X_test, y_test
X_test = torch.rand(32, 96)
X_test_reshape = X_test.view(32,96,1)
y_test = torch.rand(32, 1)

y_pred = test_model(X_test_reshape)
print(y_pred.shape)

torch.Size([32, 1])


In [30]:
loss = criterion(y_pred, y_test)

# Training loop

In [45]:
net = RNN()
# Set up MSE loss
criterion = nn.MSELoss()
optimizer = optim.Adam(
  net.parameters(), lr=0.0001
)

for epoch in range(5):
    for seqs, labels in train_loader:
        # Reshape model inputs
        seqs = seqs.view(seqs.size(0), seqs.size(1), 1)
        # Get model outputs
        outputs = net(seqs)
        # Compute loss
        labels = labels.view(-1,1)
        loss = criterion(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.032437700778245926
Epoch 2, Loss: 0.025201592594385147
Epoch 3, Loss: 0.023907987400889397
Epoch 4, Loss: 0.02443244308233261
Epoch 5, Loss: 0.026569977402687073


# Evaluating forecasting models

In [46]:
# Use create_sequences to create inputs and targets
seq_length=96
X_test, y_test = create_sequences(test_data, seq_length)
print(X_test.shape, y_test.shape)

# Create TensorDataset
dataset_test = TensorDataset(
    torch.from_numpy(X_test).float(),
    torch.from_numpy(y_test).float(),
)
print(len(dataset_test))

(34944, 96) (34944,)
34944


In [51]:
#Create dataloader with training data
test_loader = DataLoader(dataset_test, batch_size=32, shuffle=False)

In [49]:
import torchmetrics

In [52]:
from torchmetrics import MeanSquaredError
# Define MSE metric
mse = torchmetrics.MeanSquaredError()

net.eval()
with torch.no_grad():
    for seqs, labels in test_loader:
        seqs = seqs.view(seqs.size(0), seqs.size(1), 1)
        labels = labels.view(-1, 1)   # đảm bảo (batch, 1)
        outputs = net(seqs)           # (batch, 1)
        mse.update(outputs, labels)

# Compute final metric value
test_mse = mse.compute()
print(f"Test MSE: {test_mse}")


Test MSE: 0.040719348937273026


In [54]:
from torchmetrics import MeanSquaredError
# Define MSE metric
rmse = torchmetrics.MeanSquaredError(squared=False)

net.eval()
with torch.no_grad():
    for seqs, labels in test_loader:
        seqs = seqs.view(seqs.size(0), seqs.size(1), 1)
        labels = labels.view(-1, 1)   # đảm bảo (batch, 1)
        outputs = net(seqs)           # (batch, 1)
        rmse.update(outputs, labels)

# Compute final metric value
test_rmse = rmse.compute()
print(f"Test MSE: {test_rmse}")


Test MSE: 0.20179036259651184


In [56]:
import torch

all_labels = torch.cat([labels for _, labels in test_loader])
std_y = torch.std(all_labels)
mean_y = torch.mean(all_labels)

print(f"Std of target: {std_y:.4f}")
print(f"Mean of target: {mean_y:.4f}")


Std of target: 0.8046
Mean of target: 0.0252


RMSE = 0.2 > Std of target => Good result